In [1]:
# Parameters
DATE = 20211102
N_TRIALS = 50
NN_MAX_EPOCHS = 100
CNN_MAX_EPOCHS = 200
SCORING_METRIC = "f1_macro"

# Multilabel Model Training

In [2]:
import os
import pickle

import bento
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from optuna.integration import SkorchPruningCallback
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler
from sklearn.svm import SVC
from skorch import NeuralNet, NeuralNetClassifier
from skorch.callbacks import EarlyStopping, EpochScoring
from xgboost import XGBRFClassifier

In [3]:
# Other general variables
model_dir = f"../../models/ml_multilabel_{DATE}"

# Make folder to store study and model
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

## Cell x Feature Matrix

In [4]:
data = bento.io.read_h5ad("../../data/locfish/locfish_eval_20211019.h5ad")
data.shape

(10000, 1)

In [5]:
# List of features
features = [
    "cell_inner_proximity",
    "nucleus_inner_proximity",
    "nucleus_outer_proximity",
    "cell_inner_asymmetry",
    "nucleus_inner_asymmetry",
    "nucleus_outer_asymmetry",
    "l_max",
    "l_max_gradient",
    "l_min_gradient",
    "l_monotony",
    "l_half_radius",
    "point_dispersion",
    "nucleus_dispersion",
]

cell_by_feature = []
for f in features:
    cell_by_feature.append(data.to_df(f))
cell_by_feature = (
    pd.concat(cell_by_feature, axis=1).reset_index(drop=True).astype(float)
)
cell_by_feature.columns = features

## Prepare datasets

In [6]:
classes = ["cell_edge", "cytoplasmic", "none", "nuclear", "nuclear_edge"]

Split train/test features

In [7]:
X = torch.FloatTensor(cell_by_feature.values)

le = LabelBinarizer().fit(classes)
y = le.transform(data.to_df("pattern").values.flatten())
y = torch.LongTensor(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=22, stratify=y
)

Split train/test image features

In [8]:
# Display the structure of the AnnData object
print(data)

# View keys available in `obsm`, which often holds spatial coordinates
print("obsm keys:", data.obsm.keys())

# View keys available in `obs`, which often holds labels or cell metadata
print("obs keys:", data.obs.keys())

# View unstructured data keys in `uns`, in case other metadata is stored there
print("uns keys:", data.uns.keys())


AnnData object with n_obs × n_vars = 10000 × 1
    obs: 'cell_shape', 'nucleus_shape', 'batch', 'cell_radius'
    uns: 'points'
    layers: 'cell_inner_asymmetry', 'cell_inner_proximity', 'cell_outer_asymmetry', 'cell_outer_proximity', 'l_half_radius', 'l_max', 'l_max_gradient', 'l_min_gradient', 'l_monotony', 'nucleus_dispersion', 'nucleus_inner_asymmetry', 'nucleus_inner_proximity', 'nucleus_outer_asymmetry', 'nucleus_outer_proximity', 'pattern', 'point_dispersion', 'spliced', 'unspliced'
obsm keys: KeysView(AxisArrays with keys: )
obs keys: Index(['cell_shape', 'nucleus_shape', 'batch', 'cell_radius'], dtype='object')
uns keys: dict_keys(['points'])


In [9]:
print(data.uns['points'])

                 x           y  \
0        69.000000  274.000000   
1        10.000000  224.000000   
2         7.000000  172.000000   
3       138.000000  239.000000   
4       203.000000  190.000000   
...            ...         ...   
693289  159.453846  219.192167   
693290   81.871120  126.448050   
693291   16.504118   73.624290   
693292  171.830562  141.796859   
693293   74.100206   97.465173   

                                                     cell gene    pattern  \
0       mRNAlevel_10-cell2D-moderate-w1_HelaKyoto_Gapd...    0  cell_edge   
1       mRNAlevel_10-cell2D-moderate-w1_HelaKyoto_Gapd...    0  cell_edge   
2       mRNAlevel_10-cell2D-moderate-w1_HelaKyoto_Gapd...    0  cell_edge   
3       mRNAlevel_10-cell2D-moderate-w1_HelaKyoto_Gapd...    0  cell_edge   
4       mRNAlevel_10-cell2D-moderate-w1_HelaKyoto_Gapd...    0  cell_edge   
...                                                   ...  ...        ...   
693289                                             4

In [10]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.spatial import cKDTree
# import anndata  # Ensure anndata is installed
# import os

# # Load the data
# data = anndata.read_h5ad("../../data/locfish/locfish_eval_20211019.h5ad")
# print("Data loaded successfully.")

# # Extract spatial coordinates from the first two columns of `data.uns['points']`
# print("Extracting spatial coordinates...")
# spatial_coords = data.uns['points'].iloc[:, :2].to_numpy()
# print("Spatial coordinates extracted:", spatial_coords.shape)

# # Use the 'pattern' column in `data.uns['points']` as the label
# print("Extracting labels...")
# labels = data.uns['points']['pattern']
# print("Labels extracted. Unique labels:", labels.unique())

# # Define output directory
# output_dir = 'data/locfish/eval_img/'
# os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist
# print(f"Output directory set to: {output_dir}")

# # Define grid resolution and calculate bounds
# resolution = 100  # Adjust based on desired output size
# x_min, x_max = spatial_coords[:, 0].min(), spatial_coords[:, 0].max()
# y_min, y_max = spatial_coords[:, 1].min(), spatial_coords[:, 1].max()
# print("Grid bounds calculated:")
# print(f"x_min: {x_min}, x_max: {x_max}, y_min: {y_min}, y_max: {y_max}")

# # Set up grid points for rasterization
# x_grid = np.linspace(x_min, x_max, resolution)
# y_grid = np.linspace(y_min, y_max, resolution)
# grid_coords = np.array(np.meshgrid(x_grid, y_grid)).reshape(2, -1).T
# tree = cKDTree(grid_coords)
# print(f"Grid set up with resolution {resolution}. Total grid points: {len(grid_coords)}")

# # Get unique labels and assign a numeric ID to each
# unique_labels = {label: idx + 1 for idx, label in enumerate(set(labels))}
# print("Unique labels assigned IDs:", unique_labels)

# # Rasterize for each unique label and save images
# for label, label_id in unique_labels.items():
#     print(f"Rasterizing cells for label: {label} (ID: {label_id})")
#     # Initialize an empty raster grid for each label
#     raster = np.zeros((resolution, resolution))

#     # Loop through each cell and place it in the raster grid if it matches the current label
#     for coord, cell_label in zip(spatial_coords, labels):
#         if cell_label == label:
#             dist, idx = tree.query(coord)
#             x_idx, y_idx = np.unravel_index(idx, (resolution, resolution))
#             raster[x_idx, y_idx] = label_id  # Assign the label ID to the grid cell

#     # Plot and save the rasterized image for each label
#     plt.imshow(raster, cmap='viridis')
#     plt.colorbar(label='Cell Label')
#     plt.title(f"Rasterized Cell Data - Label: {label}")
#     image_path = os.path.join(output_dir, f'rasterized_cells_{label}.png')
#     plt.savefig(image_path)
#     plt.close()  # Close the figure to save memory
#     print(f"Image saved for label {label} at: {image_path}")

# print("All rasterized images saved to:", output_dir)





In [11]:
print(dir(bento.tl))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_colocation', '_composition', '_decomposition', '_flux', '_flux_enrichment', '_lp', '_neighborhoods', '_point_features', '_shape_features', 'analyze_points', 'analyze_shapes', 'coloc_quotient', 'colocation', 'comp_diff', 'decompose', 'fe', 'fe_fazal2019', 'fe_kegg', 'fe_xia2019', 'flux', 'fluxmap', 'gene_sets', 'list_point_features', 'list_shape_features', 'load_gene_sets', 'lp', 'lp_diff', 'lp_stats', 'obs_stats', 'register_point_feature', 'register_shape_feature', 'to_tensor']


In [12]:
# from torchvision import datasets, transforms

# dataset = datasets.ImageFolder(
#     "data/locfish/eval_img",
#     transform=transforms.Compose([transforms.Grayscale(), transforms.ToTensor()]),
# )
# X_img = np.array([x[0].numpy() for x in dataset])
# y_img = np.array(dataset.targets)

# le = LabelBinarizer().fit(dataset.targets)
# y_img = le.transform(y_img)

# X_img_train, X_img_test, y_img_train, y_img_test = train_test_split(
#     X_img, y_img, train_size=0.8, random_state=22, stratify=y_img
# )

# Prepare nn architectures

In [13]:
class FCModule(nn.Module):
    def __init__(
        self,
        l0_size,
        l1_size,
    ):
        super().__init__()
        fc_layers = []
        in_features = 13

        # Hidden layers
        for layer_size in [l0_size, l1_size]:
            fc_layers.extend(
                [
                    nn.Linear(in_features, layer_size),
                    nn.BatchNorm1d(layer_size),
                    nn.Dropout(0.2),
                    nn.ReLU(),
                ]
            )

            in_features = layer_size

        # Output layer
        fc_layers.append(nn.Linear(in_features, 2))
        self.model = nn.Sequential(*fc_layers)

    def forward(self, x):
        x = self.model(x.float())
        x = F.softmax(x, dim=-1)
        return x

In [14]:
def get_conv_dim(in_size, padding, dilation, kernel_size, stride):
    outsize = 1 + (in_size + 2 * padding - dilation * (kernel_size - 1) - 1) / stride
    return int(outsize)


class SpotsBinaryModule(nn.Module):
    def __init__(
        self,
        n_conv_layers,
        in_dim,
        out_channels,
        kernel_size,
        f_units_l0,
        f_units_l1,
    ) -> None:
        super().__init__()
        conv_layers = []

        in_channels = 1
        in_dim = in_dim

        # Stack (convolutions + batchnorm + activation) + maxpool
        for i in range(n_conv_layers):
            conv_layers.extend(
                [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=kernel_size,
                    ),
                    nn.BatchNorm2d(out_channels),
                    nn.Dropout(0.2),
                    nn.ReLU(),
                ]
            )

            # Compute convolved output dimensions
            in_dim = get_conv_dim(
                in_dim, padding=0, dilation=1, kernel_size=kernel_size, stride=1
            )

            in_channels = out_channels
            out_channels *= 2

        out_channels = int(out_channels / 2)

        conv_layers.append(nn.MaxPool2d(2, 2))
        in_dim = int(in_dim / 2)

        # We optimize the number of layers, hidden units and dropout ratio in each layer.
        fc_layers = [nn.Flatten()]

        # Compute flatten size
        in_features = out_channels * in_dim * in_dim
        for i in [f_units_l0, f_units_l1]:
            out_features = i
            fc_layers.extend(
                [
                    nn.Linear(in_features, out_features),
                    nn.BatchNorm1d(out_features),
                    nn.Dropout(0.2),
                    nn.ReLU(),
                ]
            )

            in_features = out_features

        fc_layers.append(nn.Linear(in_features, 2))
        self.model = torch.nn.Sequential(*[*conv_layers, *fc_layers])

    def forward(self, x):
        x = self.model(x)
        x = F.softmax(x, dim=-1)
        return x

# Optimize model hyperparameters

## Gradient-boosted Random Forest Classifier

In [15]:
def objective(trial):

    clf = OneVsRestClassifier(
        make_pipeline(
            StandardScaler(),
            XGBRFClassifier(
                n_estimators=trial.suggest_int("n_estimators", 10, 500, 10),
                max_depth=trial.suggest_int("max_depth", 2, 20),
                learning_rate=trial.suggest_float("learning_rate", 1e-4, 1, log=True),
                colsample_bytree=trial.suggest_float("colsample_bytree", 0.2, 0.8),
                reg_lambda=trial.suggest_float("reg_lambda", 1e-7, 1e-1, log=True),
                use_label_encoder=False,
                eval_metric="logloss",  # set to hide warning
            ),
        )
    )

    # Avoid overfitting with 4-fold cross validation
    return cross_val_score(
        clf, X_train.numpy(), y_train.numpy(), scoring=SCORING_METRIC, cv=4
    ).mean()

#3. Create a study object and optimize the objective function.
rf_study = optuna.create_study(
    direction="maximize",
    study_name="xgbrf",
    storage=f"sqlite:///C:/Users/Aastha_Ishan/Desktop/bento-manuscript-pub/bento-manuscript-pub/models/ml_multilabel_20211102/optuna.db",
    load_if_exists=True,
)
# rf_study = optuna.create_study()
rf_study.optimize(
    objective, n_trials=N_TRIALS, n_jobs=1, show_progress_bar=True, gc_after_trial=False
)

[I 2024-11-01 22:54:46,316] Using an existing study with name 'xgbrf' instead of creating a new one.


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2024-11-01 22:56:02,186] Trial 76 finished with value: 0.862631394514827 and parameters: {'n_estimators': 470, 'max_depth': 10, 'learning_rate': 0.06989323971429498, 'colsample_bytree': 0.6771203437258865, 'reg_lambda': 0.001108300748664384}. Best is trial 12 with value: 0.8627784636173459.
[I 2024-11-01 22:57:25,024] Trial 77 finished with value: 0.8621606506459524 and parameters: {'n_estimators': 470, 'max_depth': 10, 'learning_rate': 0.06511523255115753, 'colsample_bytree': 0.6255699707777629, 'reg_lambda': 0.04837016092125739}. Best is trial 12 with value: 0.8627784636173459.
[I 2024-11-01 22:58:46,462] Trial 78 finished with value: 0.8619980513742513 and parameters: {'n_estimators': 430, 'max_depth': 12, 'learning_rate': 0.053757459291464686, 'colsample_bytree': 0.7015924062659041, 'reg_lambda': 0.00019719200397740982}. Best is trial 12 with value: 0.8627784636173459.
[I 2024-11-01 23:00:03,849] Trial 79 finished with value: 0.8623971797208261 and parameters: {'n_estimators': 4

In [17]:
# import optuna

# optuna.delete_study(study_name="svc", storage=f"sqlite:///C:/Users/Aastha_Ishan/Desktop/bento-manuscript-pub/bento-manuscript-pub/models/ml_multilabel_20211102/optuna2.db")


In [18]:
import os
print(os.path.exists("C:/Users/Aastha_Ishan/Desktop/bento-manuscript-pub/bento-manuscript-pub/models/ml_multilabel_20211102/optuna.db"))


True


## Support Vector Classifier

In [20]:
def objective(trial):

    clf = OneVsRestClassifier(
        make_pipeline(
            StandardScaler(),
            SVC(
                kernel=trial.suggest_categorical(
                    "kernel", ["linear", "poly", "rbf", "sigmoid"]
                ),
                C=trial.suggest_float("C", 1e-5, 1, log=True),
                degree=trial.suggest_int("degree", 2, 5),
                probability=True,
            ),
        )
    )

    # Avoid overfitting with 4-fold cross validation
    return cross_val_score(clf, X_train.numpy(), y_train.numpy(), scoring=SCORING_METRIC, cv=4).mean()

# 3. Create a study object and optimize the objective function.
svc_study = optuna.create_study(
    direction="maximize",
    study_name="svc",
    storage=f"sqlite:///C:/Users/Aastha_Ishan/Desktop/bento-manuscript-pub/bento-manuscript-pub/models/ml_multilabel_20211102/optuna.db",
    load_if_exists=True,
)
svc_study.optimize(
    objective, n_trials=N_TRIALS, n_jobs=1, show_progress_bar=True, gc_after_trial=False
)

[I 2024-11-01 23:59:50,493] Using an existing study with name 'svc' instead of creating a new one.


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2024-11-02 00:00:36,047] Trial 50 finished with value: 0.8666535617503005 and parameters: {'kernel': 'rbf', 'C': 0.9335345031595607, 'degree': 5}. Best is trial 50 with value: 0.8666535617503005.
[I 2024-11-02 00:01:16,498] Trial 51 finished with value: 0.8664583204821468 and parameters: {'kernel': 'rbf', 'C': 0.9279274785235484, 'degree': 5}. Best is trial 50 with value: 0.8666535617503005.
[I 2024-11-02 00:01:57,344] Trial 52 finished with value: 0.8667399785802018 and parameters: {'kernel': 'rbf', 'C': 0.9546182446690319, 'degree': 5}. Best is trial 52 with value: 0.8667399785802018.
[I 2024-11-02 00:02:38,583] Trial 53 finished with value: 0.8666446104090747 and parameters: {'kernel': 'rbf', 'C': 0.9876545139107036, 'degree': 5}. Best is trial 52 with value: 0.8667399785802018.
[I 2024-11-02 00:03:18,412] Trial 54 finished with value: 0.8667808432013303 and parameters: {'kernel': 'rbf', 'C': 0.9801296529313217, 'degree': 5}. Best is trial 54 with value: 0.8667808432013303.
[I 20

In [21]:
print(type(y_train))
print(y_train.dtype)
y_train = y_train.long()


<class 'torch.Tensor'>
torch.int64


## Feed Forward Neural Net

In [ ]:
def objective(trial):

    clf = OneVsRestClassifier(
        NeuralNetClassifier(
            module=FCModule,
            optimizer=torch.optim.Adam,
            module__l0_size=trial.suggest_int("module__l0_size", 10, 300),
            module__l1_size=trial.suggest_int("module__l1_size", 10, 300),
            max_epochs=NN_MAX_EPOCHS,
            # Internal 80/20 train valid split
            device="cpu",
            verbose=False,
        ),
    )

    clf.fit(X_train, y_train)

    # Avoid overfitting by evaluating internal 20% validation
    return np.mean([e.history[-1, "valid_loss"] for e in clf.estimators_])

# 3. Create a study object and optimize the objective function.
nn_study = optuna.create_study(
    direction="minimize",
    study_name="ffnn",
    storage=f"sqlite:///C:/Users/Aastha_Ishan/Desktop/bento-manuscript-pub/bento-manuscript-pub/models/ml_multilabel_20211102/optuna2.db",
    load_if_exists=True,
)
nn_study.optimize(
    objective, n_trials=N_TRIALS, n_jobs=1, show_progress_bar=True, gc_after_trial=False
)

## CNN Neural Net

In [ ]:
def objective(trial):

    # Initialize classifier model
    clf = OneVsRestClassifier(
        NeuralNetClassifier(
            module=SpotsBinaryModule,
            module__n_conv_layers=2,
            module__in_dim=64,
            module__out_channels=trial.suggest_int("module__out_channels", 2, 16),
            module__kernel_size=3,
            module__f_units_l0=trial.suggest_int("module__f_units_l0", 10, 300),
            module__f_units_l1=trial.suggest_int("module__f_units_l1", 10, 100),
            optimizer=torch.optim.Adam,
            max_epochs=CNN_MAX_EPOCHS,
            callbacks=[
                EpochScoring(
                    scoring="f1_macro",
                    name="valid_f1",
                    lower_is_better=False,
                ),
            ],
            # Internal 80/20 train valid split
            device="cpu",
            verbose=False,
        ),
    )

    clf.fit(X_img_train, y_img_train)

    # Avoid overfitting by evaluating internal 20% validation
    return np.mean([e.history[-1, "valid_loss"] for e in clf.estimators_])


# 3. Create a study object and optimize the objective function.
cnn_study = optuna.create_study(
    direction="minimize",
    study_name="cnn",
    storage=f"sqlite:///C:/Users/Aastha_Ishan/Desktop/bento-manuscript-pub/bento-manuscript-pub/models/ml_multilabel_20211102/optuna2.db",
    load_if_exists=True,
)

cnn_study.optimize(
    objective, n_trials=N_TRIALS, n_jobs=1, show_progress_bar=True, gc_after_trial=False
)

# Train Models

In [22]:
rf_model = OneVsRestClassifier(
    make_pipeline(
        StandardScaler(),
        XGBRFClassifier(
            **rf_study.best_params,
            use_label_encoder=False,
            eval_metric="logloss",  # set to hide warning
        ),
    )
)

rf_model.fit(X_train.numpy(), y_train.numpy())
pickle.dump(rf_model, open(f"{model_dir}/rf_model2.pkl", "wb"))

In [23]:
svc_model = OneVsRestClassifier(
    make_pipeline(
        StandardScaler(),
        SVC(
            **svc_study.best_params,
            probability=True,
        ),
    )
)

svc_model.fit(X_train.numpy(), y_train.numpy())
pickle.dump(svc_model, open(f"{model_dir}/svc_model2.pkl", "wb"))

In [ ]:
# Train on 80%, 20% validation for early stopping
nn_model = OneVsRestClassifier(
    make_pipeline(
        StandardScaler(),
        NeuralNetClassifier(
            module=FCModule,
            **nn_study.best_params,
            callbacks=[EarlyStopping(monitor="valid_loss", patience=20)],
            optimizer=torch.optim.Adam,
            max_epochs=NN_MAX_EPOCHS,
            device="cuda",
            verbose=False,
        ),
    ),
)

nn_model.fit(X_train, y_train)
pickle.dump(nn_model, open(f"{model_dir}/nn_model2.pkl", "wb"))

In [ ]:
# Train on 80%, 20% validation for early stopping
cnn_model = OneVsRestClassifier(
    NeuralNetClassifier(
        module=SpotsBinaryModule,
        **cnn_study.best_params,
        module__n_conv_layers=2,
        module__in_dim=64,
        module__kernel_size=3,
        callbacks=[EarlyStopping(monitor="valid_loss", patience=20)],
        optimizer=torch.optim.Adam,
        max_epochs=CNN_MAX_EPOCHS,
        device="cuda",
        verbose=False,
    ),
)

cnn_model.fit(X_img_train, y_img_train)
pickle.dump(cnn_model, open(f"{model_dir}/cnn_model2.pkl", "wb"))